# Homework 5

In [1]:
import os
from getpass import getpass
os.environ['HUGGINGFACEHUB_API_TOKEN'] = getpass('Enter huggingfacehub api token: ')

Enter huggingfacehub api token:  ········


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

# load model
model = "tiiuae/falcon-7b-instruct"
# model = "tiiuae/falcon-40b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model)

falcon_pipeline = transformers.pipeline("text-generation",
                                        model=model,
                                        tokenizer=tokenizer,
                                        torch_dtype=torch.bfloat16,
                                        trust_remote_code=True,
                                        device_map="auto"
                                        )

/soft/applications/conda/2024-08-08/mconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
2024-11-18 21:10:07.644239: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731964207.665420 3552234 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731964207.671983 3552234 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-18 21:

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# load remote model
from langchain.llms import HuggingFaceHub
model = "tiiuae/falcon-7b-instruct"
# model = "tiiuae/falcon-40b-instruct"
falcon = HuggingFaceHub(
    repo_id=model,
    model_kwargs={"temperature": 0.5,
                  "max_length": 128},
)

/var/tmp/pbs.37575.sophia-pbs-01.lab.alcf.anl.gov/ipykernel_3552234/454465620.py:5: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  falcon = HuggingFaceHub(


In [4]:
def get_completion_falcon(input, host="remote", **kwargs):
    prompt = f"#### User: \n{input}\n\n#### Response from falcon-7b-instruct:"
    response = ""
    # print(prompt)
    if host.lower() == "local":
      print("invoking llm at Google Colab")
      if 'max_length' not in kwargs:
        kwargs['max_length'] = 1000

      falcon_response = falcon_pipeline(prompt,
                                      #max_length=500,
                                      do_sample=True,
                                      top_k=10,
                                      num_return_sequences=1,
                                      eos_token_id=tokenizer.eos_token_id,
                                      **kwargs,
                                      )
      response = falcon_response[0]['generated_text']

    elif host.lower() == "remote":
      print("invoking llm at Huggingface Hub")
      if "max_length" in kwargs:
        kwargs['max_new_tokens'] = kwargs['max_length']

      response = falcon.invoke(prompt, **kwargs)

    else:
      print ("invalid host value, must be 'remote' or 'local'")

    return response

In [5]:
#function for creating a random list of numbers 
import random

def get_random_number_list(num): 
    random_list = [random.randint(1, 100) for i in range(num)]
    return random_list

## Zero Shot

In [8]:
# discrete prompt
rand_list = str(get_random_number_list(8)).replace('[','').replace(']','')

prompt =  f"Please get the sum of odd numbers from this list of integers: {rand_list}"
response = get_completion_falcon(prompt)
print(response)
#this answer should be 205

invoking llm at Huggingface Hub
#### User: 
Please get the sum of odd numbers from this list of integers: 41, 47, 74, 39, 92, 11, 67, 82

#### Response from falcon-7b-instruct:

The sum of the odd numbers in the list is 137.


# few-shot

In [12]:
few_shot_prompt =  "I will provide you with a few examples of how to get the sum of odd numbers from a given list. Follow the pattern to sum the odd numbers in a given list. Example 1: List: 35, 21, 93, 83, 44, 69. Sum: 301. Example 2: List: 58, 86, 90, 73, 39, 87. Sum: 199. Now, get the sum of odd numbers from the following given list: 35, 94, 53, 76, 93, 82."

response_few_shot = get_completion_falcon(few_shot_prompt)
print(response_few_shot)
#answer should be 181                 

invoking llm at Huggingface Hub
#### User: 
I will provide you with a few examples of how to get the sum of odd numbers from a given list. Follow the pattern to sum the odd numbers in a given list. Example 1: List: 35, 21, 93, 83, 44, 69. Sum: 301. Example 2: List: 58, 86, 90, 73, 39, 87. Sum: 199. Now, get the sum of odd numbers from the following given list: 35, 94, 53, 76, 93, 82.

#### Response from falcon-7b-instruct: 
The sum of the odd numbers in the list is 275.


# chain-of-thought

In [20]:
COT_prompt = "Question: Can you sum up the odd numbers in the given list: 41, 47, 74, 39, 92, 11, 67, 82. Answer: The odd numbers in the given list are 41, 47, 39, 11, 67. 41 + 47 + 39 + 11 + 67 = 205. The answer is 205. Now, try this: Can you sum up the odd numbers in the given list: 31, 85, 83, 73, 65, 94, 32."
response_COT = get_completion_falcon(COT_prompt) 
print(response_COT)
#answer should be 337

invoking llm at Huggingface Hub
#### User: 
Question: Can you sum up the odd numbers in the given list: 41, 47, 74, 39, 92, 11, 67, 82. Answer: The odd numbers in the given list are 41, 47, 39, 11, 67. 41 + 47 + 39 + 11 + 67 = 205. The answer is 205. Now, try this: Can you sum up the odd numbers in the given list: 31, 85, 83, 73, 65, 94, 32.

#### Response from falcon-7b-instruct: 
Answer: The odd numbers in the given list are 31, 65, 94. 31 + 65 + 94 = 230. The answer is 230.


## few-shot chain-of-thought

In [23]:
few_shot_COT_prompt =  """You are very good at math.
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Examples:
Q: Sum up the odd numbers in this list: 4, 8, 9, 15, 12, 2, 1.
A: The given numbers are: (4, 8, 9, 15, 12, 2, 1). Next select the odd numbers (9, 15, 1).  Adding all the selected odd numbers (9, 15, 1) gives 25. 25 is the answer.

Q: Sum up the odd numbers in this list: 17, 10, 19, 4, 8, 12, 24.
A: The given numbers are: (17, 10, 19, 4, 8, 12, 24). Next select the odd numbers (17, 19).  Adding all the selected odd numbers (17, 19) gives 36. 36 is the answer.

Q: Sum up the odd numbers in this list: 16, 11, 14, 4, 8, 13, 24.
A: The given numbers are: (16, 11, 14, 4, 8, 13, 24). Next select the odd numbers (11, 13).  Adding all the selected odd numbers (11, 13) gives 24. 24 is the answer.

Q: Sum up the odd numbers in this list: 17, 9, 10, 12, 13, 4, 2.
A: The given numbers are: (17, 9, 10, 12, 13, 4, 2). Next select the odd numbers (17, 9, 13).  Adding all the selected odd numbers (17, 9, 13) gives 39. 39 is is the answer.

Problem:
Q: Sum up the odd numbers in this list: 15, 32, 5, 13, 82, 7, 1.
A:
"""
#15 + 5 + 13 + 7 + 1 = 41
kwargs = {"max_length": 1000}
few_shot_COT_prompt_explanation = get_completion_falcon(few_shot_COT_prompt, **kwargs)
print(few_shot_COT_prompt_explanation)



invoking llm at Huggingface Hub
#### User: 
You are very good at math.
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Examples:
Q: Sum up the odd numbers in this list: 4, 8, 9, 15, 12, 2, 1.
A: The given numbers are: (4, 8, 9, 15, 12, 2, 1). Next select the odd numbers (9, 15, 1).  Adding all the selected odd numbers (9, 15, 1) gives 25. 25 is the answer.

Q: Sum up the odd numbers in this list: 17, 10, 19, 4, 8, 12, 24.
A: The given numbers are: (17, 10, 19, 4, 8, 12, 24). Next select the odd numbers (17, 19).  Adding all the selected odd numbers (17, 19) gives 36. 36 is the answer.

Q: Sum up the odd numbers in this list: 16, 11, 14, 4, 8, 13, 24.
A: The given numbers are: (16, 11, 14, 4, 8, 13, 24). Next select the odd numbers (11, 13).  Adding all the selected odd numbers (11, 13) gives 24. 24 is the answer.

Q: Sum up the odd numbers in this list: 17, 9, 10, 12, 

## Answer

In mostly all cases, Falcon-7B-Instruct could not correctly identify the odd numbers in the given list. As a result, the answer was always wrong. It was easier to see what the thought process of the model when using the COT processes, however, this method did not lead to a result that was any more accurate than the zero- and few-shot methods. I think, if not asking about a mathematical question, the model answer would increase in quality when using the COT methods. Unfortunately, it just seems like the model could not handle the identification of odd numbers, and always offered the wrong response since it could never get this first identification step correct. 

In [ ]:
few_shot_COT_prompt =  """You are very good at math.
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Examples:
Q: Sum up the odd numbers in this list: 4, 8, 9, 15, 12, 2, 1.
A: The given numbers are: (4, 8, 9, 15, 12, 2, 1). Next select the odd numbers (9, 15, 1).  Adding all the selected odd numbers (9, 15, 1) gives 25. 25 is the answer.

Q: Sum up the odd numbers in this list: 17, 10, 19, 4, 8, 12, 24.
A: The given numbers are: (17, 10, 19, 4, 8, 12, 24). Next select the odd numbers (17, 19).  Adding all the selected odd numbers (17, 19) gives 36. 36 is the answer.

Q: Sum up the odd numbers in this list: 16, 11, 14, 4, 8, 13, 24.
A: The given numbers are: (16, 11, 14, 4, 8, 13, 24). Next select the odd numbers (11, 13).  Adding all the selected odd numbers (11, 13) gives 24. 24 is the answer.

Q: Sum up the odd numbers in this list: 17, 9, 10, 12, 13, 4, 2.
A: The given numbers are: (17, 9, 10, 12, 13, 4, 2). Next select the odd numbers (17, 9, 13).  Adding all the selected odd numbers (17, 9, 13) gives 39. 39 is is the answer.

Problem:
Q: Sum up the odd numbers in this list: 15, 32, 5, 13, 82, 7, 1.
A:
"""
#15 + 5 + 13 + 7 + 1 = 41
kwargs = {"max_length": 1000}
few_shot_COT_prompt_explanation = get_completion_falcon(few_shot_COT_prompt, **kwargs)
print(few_shot_COT_prompt_explanation)

### Playing around with temperature

In [6]:
temperature_list = [0.1, 0.2, 0.5, 0.7, 1.0, 1.5]

In [28]:
# discrete prompt
for temp in temperature_list: 
    print(f'#################### Trying temp {temp} ####################')
    prompt =  "Please get the sum of odd numbers from this list of integers: 41, 47, 74, 39, 92, 11, 67, 82"
    kwargs = {'temperature': temp}
    response = get_completion_falcon(prompt, **kwargs)
    print(response)
    #this answer should be 205

#################### Trying temp 0.1 ####################
invoking llm at Huggingface Hub
#### User: 
Please get the sum of odd numbers from this list of integers: 41, 47, 74, 39, 92, 11, 67, 82

#### Response from falcon-7b-instruct:

The sum of odd numbers from the list is 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59
#################### Trying temp 0.2 ####################
invoking llm at Huggingface Hub
#### User: 
Please get the sum of odd numbers from this list of integers: 41, 47, 74, 39, 92, 11, 67, 82

#### Response from falcon-7b-instruct:

The sum of odd numbers from the list is 2,378.
#################### Trying temp 0.5 ####################
invoking llm at Huggingface Hub
#### User: 
Please get the sum of odd numbers from this list of integers: 41, 47, 74, 39, 92, 11, 67, 82

#### Response from falcon-7b-instruct:

The sum of odd numbers from the list is 260.
#################### Trying temp 0.7 #########

In [29]:
few_shot_prompt =  "I will provide you with a few examples of how to get the sum of odd numbers from a given list. Follow the pattern to sum the odd numbers in a given list. Example 1: List: 35, 21, 93, 83, 44, 69. Sum: 301. Example 2: List: 58, 86, 90, 73, 39, 87. Sum: 199. Now, get the sum of odd numbers from the following given list: 35, 94, 53, 76, 93, 82."

for temp in temperature_list: 
    print(f'#################### Trying temp {temp} ####################')
    kwargs = {'temperature': temp}
    response_few_shot = get_completion_falcon(few_shot_prompt, **kwargs)
    print(response_few_shot)
#answer should be 181 

#################### Trying temp 0.1 ####################
invoking llm at Huggingface Hub
#### User: 
I will provide you with a few examples of how to get the sum of odd numbers from a given list. Follow the pattern to sum the odd numbers in a given list. Example 1: List: 35, 21, 93, 83, 44, 69. Sum: 301. Example 2: List: 58, 86, 90, 73, 39, 87. Sum: 199. Now, get the sum of odd numbers from the following given list: 35, 94, 53, 76, 93, 82.

#### Response from falcon-7b-instruct:

The sum of odd numbers from the given list is 301.
#################### Trying temp 0.2 ####################
invoking llm at Huggingface Hub
#### User: 
I will provide you with a few examples of how to get the sum of odd numbers from a given list. Follow the pattern to sum the odd numbers in a given list. Example 1: List: 35, 21, 93, 83, 44, 69. Sum: 301. Example 2: List: 58, 86, 90, 73, 39, 87. Sum: 199. Now, get the sum of odd numbers from the following given list: 35, 94, 53, 76, 93, 82.

#### Response fro

In [30]:
COT_prompt = "Question: Can you sum up the odd numbers in the given list: 41, 47, 74, 39, 92, 11, 67, 82. Answer: The odd numbers in the given list are 41, 47, 39, 11, 67. 41 + 47 + 39 + 11 + 67 = 205. The answer is 205. Now, try this: Can you sum up the odd numbers in the given list: 31, 85, 83, 73, 65, 94, 32."
    
for temp in temperature_list: 
    print(f'#################### Trying temp {temp} ####################')
    kwargs = {'temperature': temp}
    response_COT = get_completion_falcon(COT_prompt, **kwargs) 
    print(response_COT)
#answer should be 337

#################### Trying temp 0.1 ####################
invoking llm at Huggingface Hub
#### User: 
Question: Can you sum up the odd numbers in the given list: 41, 47, 74, 39, 92, 11, 67, 82. Answer: The odd numbers in the given list are 41, 47, 39, 11, 67. 41 + 47 + 39 + 11 + 67 = 205. The answer is 205. Now, try this: Can you sum up the odd numbers in the given list: 31, 85, 83, 73, 65, 94, 32.

#### Response from falcon-7b-instruct:

The odd numbers in the given list are 31, 85, 83, 73, 65, 94. The sum of these odd numbers is 205.
#################### Trying temp 0.2 ####################
invoking llm at Huggingface Hub
#### User: 
Question: Can you sum up the odd numbers in the given list: 41, 47, 74, 39, 92, 11, 67, 82. Answer: The odd numbers in the given list are 41, 47, 39, 11, 67. 41 + 47 + 39 + 11 + 67 = 205. The answer is 205. Now, try this: Can you sum up the odd numbers in the given list: 31, 85, 83, 73, 65, 94, 32.

#### Response from falcon-7b-instruct:

The sum of the 

In [9]:
few_shot_COT_prompt =  """You are very good at math.
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Examples:
Q: Sum up the odd numbers in this list: 4, 8, 9, 15, 12, 2, 1.
A: The given numbers are: (4, 8, 9, 15, 12, 2, 1). Next select the odd numbers (9, 15, 1).  Adding all the selected odd numbers (9, 15, 1) gives 25. 25 is the answer.

Q: Sum up the odd numbers in this list: 17, 10, 19, 4, 8, 12, 24.
A: The given numbers are: (17, 10, 19, 4, 8, 12, 24). Next select the odd numbers (17, 19).  Adding all the selected odd numbers (17, 19) gives 36. 36 is the answer.

Q: Sum up the odd numbers in this list: 16, 11, 14, 4, 8, 13, 24.
A: The given numbers are: (16, 11, 14, 4, 8, 13, 24). Next select the odd numbers (11, 13).  Adding all the selected odd numbers (11, 13) gives 24. 24 is the answer.

Q: Sum up the odd numbers in this list: 17, 9, 10, 12, 13, 4, 2.
A: The given numbers are: (17, 9, 10, 12, 13, 4, 2). Next select the odd numbers (17, 9, 13).  Adding all the selected odd numbers (17, 9, 13) gives 39. 39 is is the answer.

Problem:
Q: Sum up the odd numbers in this list: 15, 32, 5, 13, 82, 7, 1.
A:
"""
#15 + 5 + 13 + 7 + 1 = 41
for temp in temperature_list: 
    print(f'#################### Trying temp {temp} ####################')
    kwargs = {'temperature': temp, 
              "max_length": 1000}
    few_shot_COT_prompt_explanation = get_completion_falcon(few_shot_COT_prompt, **kwargs)
    print(few_shot_COT_prompt_explanation)



#################### Trying temp 0.1 ####################
invoking llm at Huggingface Hub
#### User: 
You are very good at math.
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Examples:
Q: Sum up the odd numbers in this list: 4, 8, 9, 15, 12, 2, 1.
A: The given numbers are: (4, 8, 9, 15, 12, 2, 1). Next select the odd numbers (9, 15, 1).  Adding all the selected odd numbers (9, 15, 1) gives 25. 25 is the answer.

Q: Sum up the odd numbers in this list: 17, 10, 19, 4, 8, 12, 24.
A: The given numbers are: (17, 10, 19, 4, 8, 12, 24). Next select the odd numbers (17, 19).  Adding all the selected odd numbers (17, 19) gives 36. 36 is the answer.

Q: Sum up the odd numbers in this list: 16, 11, 14, 4, 8, 13, 24.
A: The given numbers are: (16, 11, 14, 4, 8, 13, 24). Next select the odd numbers (11, 13).  Adding all the selected odd numbers (11, 13) gives 24. 24 is the answer

## Answer

Temperature controls the randomness of the generated responses. I notice that the responses get more random and creative with higher temperature. 

- Low Temperature: The model is more focused on the problem and generates a more expected response, like giving you a sum that it came up with. 
- High Temperature: The model generates more creative responses that are very unexpected. For example, at the last high temperature, the response is no longer a sum but an entire block of text in another language. 